In [68]:
import os
import re
import pandas as pd

# Limpeza dos discursos

## 1 Criando uma lista com todos os discursos

In [57]:
# file = open('discursos/nI1nQ0nS011nO0.txt', 'r')
# text = file.read()
# file.close
# text

In [69]:
discursos = []
for i in os.listdir('discursos'):
    file = open(f'discursos/{i}', 'r')
    text = file.read()
    discursos.append(text)
    file.close  


In [70]:
discursos = pd.Series(discursos)

In [61]:
#Criando lista com todos os conectivos da lingua portuguesa
conectivos = pd.read_html('https://duvidas.dicio.com.br/conectivos/')

a = conectivos[0]['Exemplos:']
conectivos_lista = []
for i in a:
    conectivos_lista.append(i)
conectivos_lista = " ".join(conectivos_lista)

conectivos_lista = conectivos_lista.replace('; ', ' ').replace('… ', ' ').split(' ')
conectivos_lista = list(dict.fromkeys(conectivos_lista))


## 2 Verificando se houve revisão nos discursos

In [71]:
revisado = []
for i in range(len(discursos)):
    if(re.search(r'Sem revisão', discursos[i])):
        revisado.append(False)
    else:
        revisado.append(True)
revisao = pd.Series(revisado)

In [65]:
discursos = pd.Series(df_discursos)


df_discursos = {"discursos" : discursos,
                "revisao" : revisao}
df_discursos = pd.concat(df_discursos, axis=1)